In [2]:
#Esta es una version de gr^G(M) donde L_{-1} contribuye con 1 (no con 2) y hay "derivada parcial" (una variante digamos)
var('q t')
#pari.allocatemem(8589934592)
pari.allocatemem(9589934592)

#suma de vectores en M(1/2,1/2) y gr^G(M(1/2,1/2)), representados como diccionario
def vector_sum(u, v):
    return dict((x, u.get(x, 0) + v.get(x, 0)) for x in set(u)|set(v))

#simplifica vectores de M(1/2,1/2) representados como diccionario desconsiderando orden y botando los que
#tienen coeficiente 0
def vector_simpl(u):
    v = {}
    S = set()
    for a in set(u):
        b = sorted(a) #esta parte realmente es necesaria
        b.reverse()
        b = tuple(b)
        S.add(b)
        v = vector_sum(v, {b:u[a]})
    return {a:v[a] for a in S if v[a] != 0}

def remove_zeros(u):
    return {a:u[a] for a in set(u) if u[a] != 0}

#virasoro en U(Vir(1/2)), version preliminar
def univ_vector_simpl1(a):
    for i in range(len(a)-1):
        if a[i]<a[i+1]:
            return {a[0:i]+(a[i]+a[i+1],)+a[i+2:len(a)]:a[i+1]-a[i], a[0:i]+(a[i+1],)+(a[i],)+a[i+2:len(a)]:1}
    return {a:1}

#virasoro en U(Vir(1/2,1/2))
def univ_vector_simpl(u):
    v={}
    for a in set(u):
        w = univ_vector_simpl1(a)
        v = vector_sum(v, {b:w[b]*u[a] for b in set(w)}) #esto lo hace ligeramente mas rapido
    if v == u:
        return v
    return univ_vector_simpl(v)

#producto de escalar por vector en gr^G(M(1/2,1/2))
def scalar_times_vector(u, v):
    w = {}
    for a in set(u):
        for b in set(v):
            w = vector_sum(w, {a+b:u[a]*v[b]})
    return vector_simpl(w)

def truncate_vector(u, p):
    assume(x>3)
    print(sum(list(set(u))[0]))
    n = sum(list(set(u))[0])
    w = {}
    for a in set(u):
        if weight(a)>=p:
            if a[len(a)-3:len(a)]==(1,1,1):
                w = vector_sum(w, {a[0:len(a)-3]+(3,):-3*u[a]})
            elif a[len(a)-2:len(a)]==(1,1):
                w = vector_sum(w, {a[0:len(a)-2]+(2,):-4/3*u[a]})
            elif a[len(a)-2:len(a)]==(2,1):
                w = vector_sum(w, {a[0:len(a)-2]+(3,):-5/4*u[a]})
            elif True:
                w = vector_sum(w, {a:u[a]})
    c0 = ordered_partitions1(n, p)
    w0 = [{tuple(c0[t]):u[tuple(c0[t])] for t in range(len(c0)) if tuple(c0[t]) in set(u)} for u in w]
    c1 = ordered_partitions1(n, p+1)
    w1 = [{tuple(c1[t]):u[tuple(c1[t])] for t in range(len(c1)) if tuple(c1[t]) in set(u)} for u in w]
    c1 = ordered_partitions1(n, p+2)
    w1 = [{tuple(c2[t]):u[tuple(c2[t])] for t in range(len(c2)) if tuple(c2[t]) in set(u)} for u in w]
    ww = w0
    ww = vector_sum(ww, w1)
    ww = vector_sum(ww, w2)
    return ww

def truncate_vectorv2(u, p):
    w = {}
    for a in set(u):
        if weight(a)>=p:
            if a[len(a)-3:len(a)]==(1,1,1):
                w = vector_sum(w, {a[0:len(a)-3]+(3,):-3*u[a]})
            elif a[len(a)-2:len(a)]==(1,1):
                w = vector_sum(w, {a[0:len(a)-2]+(2,):-4/3*u[a]})
            elif a[len(a)-2:len(a)]==(2,1):
                w = vector_sum(w, {a[0:len(a)-2]+(3,):-5/4*u[a]})
            elif True:
                w = vector_sum(w, {a:u[a]})
    return w

#producto de escalar por vector en gr^G(M(1/2,1/2))
def constant_times_vector(t, u):
    return {a:t*u[a] for a in set(u)}

#aplicar varias veces virasoro
def several_virasoro(a, u):
    return univ_vector_simpl({a+b:u[b] for b in set(u)})

#particiones de n
def Partitions1(n):
    if n < 0:
        return [[]]
    a=list(Partitions(n))
    a.reverse() #al parecer, esto mejora performance en un 12% masomenos, talvez otros ordenes son mejores aun
    return a

#particiones de n que no terminan en 1
def Partitions2(n):
    if n == 0:
        return [[]]
    a = []
    b = Partitions1(n)
    for i in range(len(b)):
        if b[i][len(b[i])-1] > 1:
            a = a + [b[i]]
    return a

def ordered_partitions1(n, p):
    if n < 0:
        return [[]]
    a = Partitions1(n)
    inp = ''
    w = [a[i] for i in range(len(a)) if weight(a[i])==p]
    for i in range(len(a)):
        if weight(a[i])==p:
            inp = inp+prt2(a[i]) + ','
    inp = inp[:-1]
    st = 'option(noredefine);'
    st = st + 'ring R = QQ, (x(2..' + str(n) + ')), (dp, C);'
    st = st + 'module M =' + inp + ';'
    st = st + 'intvec resultt1 = sort(M)[2];'
    st = st + 'intvec resultt = 0:size(resultt1);'
    st = st + 'for(int i=1; i<=size(resultt); i++){resultt[i]=resultt1[size(resultt)+1-i]-1;}'
    st = st + 'resultt;'
    st = singular.eval(st)
    b = eval(st)
    ww = [w[b[i]] for i in range(len(b))]
    return ww

def ordered_Par2titions1(n):
    st = 'option(noredefine);'
    st = st + 'ring R = QQ, (x(2..' + str(n) + ')), (dp, C);'
    st = st + 'module M =' + prt_n(n) + ';'
    st = st + 'intvec resultt1 = sort(M)[2];'
    st = st + 'intvec resultt = 0:size(resultt1);'
    st = st + 'for(int i=1; i<=size(resultt); i++){resultt[i]=resultt1[size(resultt)+1-i]-1;}'
    st = st + 'resultt;'
    st = singular.eval(st)
    b = eval(st)
    return b

def newPartitions1(n):
    if n < 0:
        return [[]]
    if n == 0:
        return [[]]
    if n == 1:
        return [[1]]
    if n == 2:
        return [[2],[1,1]]
    if n == 3:
        return [[2,1],[1,1,1],[3]]
    w = []
    for i in range(n,3,-1):
        w = w + ordered_partitions1(n, i)
    w = w + [[n-1,1], [n]]
    return w

#particiones de n que no terminan en 2 ni 1
def Partitions3(n):
    if n == 0:
        return [[]]
    a = []
    b = Partitions1(n)
    for i in range(len(b)):
        if b[i][len(b[i])-1] > 2:
            a = a + [b[i]]
    return a

def Partitions_one1(n):
    if n == 0:
        return [] #aqui tiene que ser vacio
    a = []
    b = Partitions1(n)
    for i in range(len(b)):
        if b[i] == [1]:
            a = a + [[1]]
        if number_of_ones(b[i])==1 and b[i][len(b[i])-2] > 2:
            a = a + [b[i]]
    return a

def Partitions_1_1(n):
    if n == 0:
        return [] #aqui tiene que ser vacio
    a = []
    b = Partitions1(n)
    for i in range(len(b)):
        if b[i] == [1,1]:
            a = a + [[1,1]]
        if number_of_ones(b[i])==2 and b[i][len(b[i])-3] > 3:
            a = a + [b[i]]
    return a

#true si a contiene a b y false de lo contrario
def is_subpartition(a,b):
    if len(a)<len(b):
        return false
    for i in range(len(a)-len(b)+1):
        if a[i:i+len(b)] == b:
            return true
    return false

#dice si debe saltar la particion
def missing_partition_final(a):
    if a == []:
        return false
    if is_subpartition(a,[2]) or is_subpartition(a,[3,3]) or is_subpartition(a,[4,3,1]) or is_subpartition(a,[4,4,1]) or is_subpartition(a,[6,5,3,1]):
        return true
    if len(a)>2:
        for i in range(len(a)-2):
            if a[i]-a[i+2]<=2:
                return true
    if len(a)>3:
        for i in range(len(a)-3):
            if a[i]-a[i+3]<=3 or (a[i]-a[i+3]<=4 and a[i+1]-a[i+2]<=3):
                return true
    if len(a)>4:
        for i in range(len(a)-4):
            if a[i]-a[i+1]==1 and a[i+1]-a[i+2]==2 and a[i+2]-a[i+3]==2 and a[i+3]-a[i+4]==1:
                return true
    return false

def missing_partitions_final(n):
    a = newPartitions1(n)
    pivs = [i for i in range(len(a)) if missing_partition_final(a[i])]
    return pivs

#dice si debe saltar la particion
def missing_partition(a):
    if a == []:
        return false
    if len(a)>1:
        if a[len(a)-2:len(a)]==[3,3]:
            return true
    if len(a)>2:
        for i in range(len(a)-2):
            if a[i]-a[i+2]<=2:
                return true
    if len(a)>3:
        for i in range(len(a)-3):
            if a[i]-a[i+3]<=3 or (a[i]-a[i+3]<=4 and a[i+1]-a[i+2]<=3):
                return true
    if len(a)>4:
        for i in range(len(a)-4):
            if a[i]-a[i+1]==1 and a[i+1]-a[i+2]==2 and a[i+2]-a[i+3]==2 and a[i+3]-a[i+4]==1:
                return true
    return false

def missing_partition_one1(a):
    a = a[:-1]
    if a == []:
        return false
    if len(a)>1:
        if (a[len(a)-2:len(a)]==[3,3] or a[len(a)-2:len(a)]==[4,3] or a[len(a)-2:len(a)]==[4,4]):
            return true
    if len(a)>2:
        if a[len(a)-3:len(a)]==[6,5,3]:
            return true
        for i in range(len(a)-2):
            if a[i]-a[i+2]<=2:
                return true
    if len(a)>3:
        for i in range(len(a)-3):
            if a[i]-a[i+3]<=3 or (a[i]-a[i+3]<=4 and a[i+1]-a[i+2]<=3):
                return true
    if len(a)>4:
        for i in range(len(a)-4):
            if a[i]-a[i+1]==1 and a[i+1]-a[i+2]==2 and a[i+2]-a[i+3]==2 and a[i+3]-a[i+4]==1:
                return true
    return false

def missing_partition_1_1(a):
    a = a[:-2]
    if a == []:
        return false
    if len(a)>1:
        if (a[len(a)-2:len(a)]==[3,3] or a[len(a)-2:len(a)]==[4,3] or a[len(a)-2:len(a)]==[4,4] or a[len(a)-2:len(a)]==[5,4]):
            return true
    if len(a)>2:
        if a[len(a)-3:len(a)]==[6,5,3]:
            return true
        for i in range(len(a)-2):
            if a[i]-a[i+2]<=2:
                return true
    if len(a)>3:
        for i in range(len(a)-3):
            if a[i]-a[i+3]<=3 or (a[i]-a[i+3]<=4 and a[i+1]-a[i+2]<=3):
                return true
    if len(a)>4:
        for i in range(len(a)-4):
            if a[i]-a[i+1]==1 and a[i+1]-a[i+2]==2 and a[i+2]-a[i+3]==2 and a[i+3]-a[i+4]==1:
                return true
    return false

#numero de L_{-n} que aparece en una particion con n>=2
def number_greater2(a):
    k = 0
    if a == ():
        return 0
    for i in range(len(a)):
        if a[i]>1:
            k = k + 1
        else:
            return k
    return k

#numero de L_{-1} que aparece en una particion
def number_of_ones(a):
    k = 0
    for i in range(len(a)):
        if a[i]==1:
            k = k+1
    return k

#particiones de n en 0, 1, 2 partes
def Partitions1_into_parts(n):
    a = Partitions1(n)
    b = [[]]*(n+1)
    for i in range(len(a)):
        m = 2*(len(a[i])-number_of_ones(a[i]))+number_of_ones(a[i])
        b[m]= b[m]+[i]
    return b

#normaliza las lineas de la matriz haciendo que el primer coeficiente no zero sea 1
def simplify_vector(u):
    for i in range(len(u)):
        if u[i] != 0:
            return u/u[i]
        
#pivotes de la matriz 
def leading_matrix(A):
    A = A.echelon_form()
    a = []
    i = 0
    for j in range(A.ncols()):
        if i < A.nrows():
            if A[i,j] != 0:
                a = a + [j]
                i += 1      
    return a

#quita las lineas 0 de una matriz
def remove_zero_rows(A):
    for i in range(A.nrows()):
        if tuple(A[i]) == (0,)*(A.ncols()):
            A = A.delete_rows(range(i,A.nrows()))
            return A
    return A

#interseccion de dos subespacios vectoriales
def intersection(U, V):
    m = U.ncols()
    W = matrix(pari.mattranspose(pari.matintersect(pari(U.transpose()), pari(V.transpose()))))
    if W.nrows()==0:
        return matrix(0, m)
    return W

#inserta una linea en una matriz
def insert_row(A, k, a):
    return matrix(A.rows()[:k] + [a] + A.rows()[k:])

#matriz base del subespacio de U(Vir(1/2,1/2))_n generado por los vectores singular m1 y m2 en relacion al orden dado por Partitions1
def newbase_subspace(n):
    m1 = {(1,1):3, (2,):-4}
    #m2 = {(1,1,1):5, (2,1):-12}
    # m2 = {(1,1,1):1, (3,):-3}
    # m2 = {(1,1,1):5, (2,1):-12}
    m2 = {(3,):5, (2,1):-4}
    # a = Partitions1(n)
    # b = Partitions1(n-2)
    # c = Partitions1(n-3)
    a = newPartitions1(n)
    b = newPartitions1(n-2)
    c = newPartitions1(n-3)
    bm1 = [several_virasoro(tuple(b[i]), m1) for i in range(len(b))]
    cm2 = [several_virasoro(tuple(c[i]), m2) for i in range(len(c))]
    A = matrix({(i,j): bm1[i].get(tuple(a[j]), 0) for j in range(len(a)) for i in range(len(b))}, sparse = False)
    B = matrix({(i,j): cm2[i].get(tuple(a[j]), 0) for j in range(len(a)) for i in range(len(c))}, sparse = False)
    A = A.stack(B)
    A = A.rref()
    A = remove_zero_rows(A)
    return A

#matriz base del subespacio de U(Vir(1/2,1/2))_n generado por los vectores singular m1 y m2 en relacion al orden dado por Partitions1
def base_subspace(n):
    m1 = {(1,1):3, (2,):-4}
    #m2 = {(1,1,1):5, (2,1):-12}
    # m2 = {(1,1,1):1, (3,):-3}
    # m2 = {(1,1,1):5, (2,1):-12}
    m2 = {(3,):5, (2,1):-4}
    # a = Partitions1(n)
    # b = Partitions1(n-2)
    # c = Partitions1(n-3)
    a = Partitions1(n)
    b = Partitions1(n-2)
    c = Partitions1(n-3)
    bm1 = [several_virasoro(tuple(b[i]), m1) for i in range(len(b))]
    cm2 = [several_virasoro(tuple(c[i]), m2) for i in range(len(c))]
    A = matrix({(i,j): bm1[i].get(tuple(a[j]), 0) for j in range(len(a)) for i in range(len(b))}, sparse = False)
    B = matrix({(i,j): cm2[i].get(tuple(a[j]), 0) for j in range(len(a)) for i in range(len(c))}, sparse = False)
    A = A.stack(B)
    A = A.rref()
    A = remove_zero_rows(A)
    return A

def base_subspace2(n, b, c):
    m1 = {(1,1):3, (2,):-4}
    #m2 = {(1,1,1):5, (2,1):-12}
    # m2 = {(1,1,1):1, (3,):-3}
    # m2 = {(1,1,1):5, (2,1):-12}
    # m2 = {(3,):5, (2,1):-4}
    a = Partitions1(n)
    bm1 = [several_virasoro(tuple(b[i]), m1) for i in range(len(b))]
    cm2 = [several_virasoro(tuple(c[i]), m2) for i in range(len(c))]
    A = matrix({(i,j): bm1[i].get(tuple(a[j]), 0) for j in range(len(a)) for i in range(len(b))}, sparse = False)
    B = matrix({(i,j): cm2[i].get(tuple(a[j]), 0) for j in range(len(a)) for i in range(len(c))}, sparse = False)
    A = A.stack(B)
    A = A.rref()
    A = remove_zero_rows(A)
    return A

#todas las particiones hasta n
def parts_up_to(n):
    return [Partitions1(i) for i in range(n+1)]

#todas las particiones que cterminan en 2 hasta n
def parts2_up_to(n):
    return [Partitions2(i) for i in range(n+1)]

#todas las particiones que cterminan en 2 hasta n
def parts3_up_to(n):
    return [Partitions3(i) for i in range(n+1)]

#expande en potencias t^p*q^n
def expansion_t_q(n):
    a = Partitions1(n)
    b = Partitions1_into_parts(n)
    A = base_subspace(n)
    filter_list = list(leading_matrix(A))
    m = range(len(a))
    c = [x for x in m if x not in filter_list]
    B = matrix(0, len(c))
    d = [-1]*len(a)
    for i in range(len(filter_list)):
        d[filter_list[i]]=i
    k = 0
    for i in range(len(b)):
        for j in range(len(b[i])):
            if d[b[i][j]]!=-1:
                row = [A[d[b[i][j]], c[s]] for s in range(len(c))]
                B = insert_row(B, B.nrows(), row)
            else:
                row = [0]*len(c)
                for r in range(len(c)): #esta parte esta medio rara pero bueno, talvez meter un diccionario seria bueno
                    if c[r]==b[i][j]:
                        row[r]=1
                B = insert_row(B, B.nrows(), row)         
        B = B.echelon_form()
        B = remove_zero_rows(B)
        print((i,B.nrows()-k))
        k = B.nrows()

#imprime para colocar en singular
def prt2(a):
    st = ""
    m = number_of_ones(a)
    for i in range(len(a)-m):
        st = st + "x(" + str(a[i]) + ")*"
    st = st + "gen(" + str(m+1) + ")"
    return st

#imprime para colocar en singular
def prt1(u):
    st = ""
    for a in set(u):
        st = st + str(u[a]) + "*" + prt2(a) + "+"
    st = st[:-1]
    return st

#inprime para colocar en singular
def prt(w):
    st = ""
    for i in range(len(w)):
        st = st + prt1(w[i]) + ","
    st = st[:-1]
    return st

#transforma en 0 vectores que tienen mas de un uno
def three_ones_equals_zero(u): #esto talvez se pueda mejorar
    return {a:u[a] for a in set(u) if number_of_ones(a) < 3}

#matriz del kernel de gr^G(f), sin simplificaciones
def subspace_matrix(n): 
    w = []
    q = parts_up_to(n)
    for m in range(n+1):
        a = q[m]
        A = base_subspace(m)
        B = matrix(0, len(a))
        for i in range(m+1):
            b = Partitions1_into_parts(m)[i]
            B2 = matrix(len(b), len(a))
            for j in range(len(b)):
                B2[j, b[j]] = 1
            B = B.stack(B2)    
            C = intersection(B, A) # esto es mas rapido que C = intersection(A, B)
            #C = intersection(A, B)
            w.extend([remove_zeros({tuple(a[b[t]]):C[j,b[t]] for t in range(len(b))}) for j in range(C.nrows())])
    ww = [u for u in w if u != {}]
    return ww

def subspace_matrix2(n, p, A):
    w = []
    q = parts_up_to(n)
    a = Partitions1(n)
    B = matrix(0, len(a))
    b = []
    for i in range(p+1):
        b = Partitions1_into_parts(n)[i]
        B2 = matrix(len(b), len(a))
        for j in range(len(b)):
            B2[j, b[j]] = 1
        B = B.stack(B2)    
    C = intersection(B, A)
    w.extend([remove_zeros({tuple(a[b[t]]):C[j,b[t]] for t in range(len(b))}) for j in range(C.nrows())])
    ww = [u for u in w if u != {}]
    return ww

def subspace_matrix3(n, p, A):
    w = []
    q = parts_up_to(n)
    a = Partitions1(n)
    B = matrix(0, len(a))
    b = []
    for i in range(p+1):
        b = Partitions1_into_parts(n)[i]
        B2 = matrix(len(b), len(a))
        for j in range(len(b)):
            B2[j, b[j]] = 1
        B = B.stack(B2)    
    C = intersection(B, A)
    w.extend([remove_zeros({tuple(a[b[t]]):C[j,b[t]] for t in range(len(b))}) for j in range(C.nrows())])
    ww = [u for u in w if u != {}]
    c = ordered_partitions1(n, p)
    www = [{tuple(c[t]):u.get(tuple(c[t]),0) for t in range(len(c))} for u in ww]
    #www = [remove_zeros({tuple(c[t]):u.get(tuple(c[t]),0) for t in range(len(c))}) for u in ww]
    return www

def subspace_matrix4(n, p, A):
    w = []
    q = parts_up_to(n)
    a = Partitions1(n)
    B = matrix(0, len(a))
    b = []
    for i in range(p+1):
        b = Partitions1_into_parts(n)[i]
        B2 = matrix(len(b), len(a))
        for j in range(len(b)):
            B2[j, b[j]] = 1
        B = B.stack(B2)    
    C = intersection(B, A)
    w.extend([remove_zeros({tuple(a[b[t]]):C[j,b[t]] for t in range(len(b))}) for j in range(C.nrows())])
    ww = [u for u in w if u != {}]
    c = ordered_partitions1(n, p)
    www = [{tuple(c[t]):u[tuple(c[t])] for t in range(len(c)) if tuple(c[t]) in set(u)} for u in ww]
    #www = [remove_zeros(u) for u in www]
    return www

#transforma un polinomio en particion
def to_partition(a):
    b = []
    max = 0
    while(a[max] == 0):
        max = max + 1
    i = len(a)-2
    while(i!=max-1):
        b = [len(a)-i]*a[i]+b
        i = i-1
    if a[len(a)-1]==3:
        b = b + [1,1]
    if a[len(a)-1]==2:
        b = b + [1]
    return b

#transforma una particion en polinomio
def to_polynomial(a,n):
    b = [0]*n
    b[n-1] = number_of_ones(a)
    for i in range(len(a)):
        if a[i]!= 1:
            b[a[i]-2] = b[a[i]-2]+1
    return b        

#da los exponentes que se deben evitar en forma de polinomio
def avoid_up_to(n):
    w = subspace_matrix(n)
    st = 'option(noredefine);'
    st = st + 'ring R = QQ, (x(2..' + str(n) + ')), (dp, C);'
    st = st + 'module M =' + prt(w) + ';'
    st = st + 'module G = std(M);'
    st = st + 'string result = "";'
    st = st + 'for(int i = 1; i<=size(G); i++){result = result + "[" + string(leadexp(G[i])) + "],";}'
    st = st + 'result = result[1..size(result)-1];'
    st = st + 'result = "[" + result + "]";'
    st = st + 'result;'
    st = singular.eval(st)
    avoid = eval(st)
    for i in range(len(avoid)):
        avoid[i][n-1] = avoid[i][n-1]-1
    return avoid

#da los exponentes que se deben evitar pero escritos en forma de particion
def avoid_up_to_v0(n):
    w = subspace_matrix(n)
    st = 'option(noredefine);'
    st = st + 'ring R = QQ, (x(' + str(n) + '..2)), (dp, C);' 
    st = st + 'module M =' + prt(w) + ';'
    st = st + 'module G = std(M);'
    st = st + 'string result = "";'
    st = st + 'for(int i = 1; i<=size(G); i++){result = result + "[" + string(leadexp(G[i])) + "],";}'
    st = st + 'result = result[1..size(result)-1];'
    st = st + 'result = "[" + result + "]";'
    st = st + 'result;'
    st = singular.eval(st)
    b = eval(st)
    avoid = [[1,1,1]]
    for i in range(len(b)):
        avoid = avoid + [to_partition(b[i])]
    return avoid

#dice si p divide a q donde p y q tienen n elementos
def p_divides_q(p,q,n):
    for i in range(n):
        if p[i]>q[i]:
            return false
    return true

def a_contains_any_of(a, avoid, n):
    p = to_polynomial(a, n)
    if p[n-1]>2:
        return true
    for i in range(len(avoid)):
        if p[n-1] == avoid[i][n-1] and p_divides_q(avoid[i][:-1], p[:-1], n-1):
            return true
    return false

#calcula la base de gr^G(M)/ker hasta n
def base_up_to(n): #esto a veces hasta demora menos que avoid_up_to
    base = [[[]], [[1]], [[1,1]]] #aqui la cague
    avoid = avoid_up_to(n)
    q = parts_up_to(n)
    for i in range(3, n+1):
        add = []
        a = q[i]
        for j in range(len(a)):
            if not a_contains_any_of(a[j], avoid, n):
                add = add + [a[j]]
        base = base + [add]
    return base

def base_maybe_up_to(n): #esto a veces hasta demora menos que avoid_up_to
    base = []
    for i in range(n+1):
        a = Partitions1(i)
        add = [a[j] for j in range(len(a)) if not missing_partition_final(a[j])]
        base = base + [add]
    return base

def base_maybe_up_to_g2(n):
    base = []
    #q = parts3_up_to(n)
    for i in range(n+1):
        #add = []
        a = Partitions3(i)
        add = [a[j] for j in range(len(a)) if not missing_partition(a[j])]
        # for j in range(len(a)):
        #     if not missing_partition(a[j]):
        #         add = add + [a[j]]
        base = base + [add]
    return base

def base_maybe_up_to_1(n):
    base = []
    #q = parts3_up_to(n)
    for i in range(n+1):
        #add = []
        a = Partitions_one1(i)
        add = [a[j] for j in range(len(a)) if not missing_partition_one1(a[j])]
        # for j in range(len(a)):
        #     if not missing_partition(a[j]):
        #         add = add + [a[j]]
        base = base + [add]
    return base

def base_maybe_up_to_1_1(n):
    base = []
    #q = parts3_up_to(n)
    for i in range(n+1):
        #add = []
        a = Partitions_1_1(i)
        add = [a[j] for j in range(len(a)) if not missing_partition_1_1(a[j])]
        # for j in range(len(a)):
        #     if not missing_partition(a[j]):
        #         add = add + [a[j]]
        base = base + [add]
    return base

def weight(a):
    return 2*(len(a)-number_of_ones(a))+number_of_ones(a)

#la serie en t^p*q^n que sale de la base de base_up_to
def series_t_q(w):
    series = 0
    for i in range(len(w)):
        series_t = 0
        for j in range(len(w[i])):
            series_t = series_t + t^weight(w[i][j])
        series = series + series_t*q^i
    return series

#saca los que terminan en 1 1  en la base de base_up_to
def A_1_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if number_of_ones(w[i][j])==2:
                a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if number_of_ones(w[i][j])==1:
                a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if number_of_ones(w[i][j])==0:
                a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 0:
                if w[i][j][len(w[i][j])-1]==3:
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_4(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 0:
                if w[i][j][len(w[i][j])-1]==4:
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g4(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if number_of_ones(w[i][j])==0:
                if len(w[i][j])==0:
                    a = a + [w[i][j]]
                else:
                    if w[i][j][len(w[i][j])-1]>4:
                        a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if number_of_ones(w[i][j])==0:
                if len(w[i][j])==0:
                    a = a + [w[i][j]]
                else:
                    if w[i][j][len(w[i][j])-1]>3:
                        a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_4_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]==4):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g4_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [3]:
                a = a + [[3]]
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]>4):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_5_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]==5):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g5_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [3]:
                a = a + [[3]]
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]>5):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_4_4(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==4 and w[i][j][len(w[i][j])-2]==4):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_5_4(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==4 and w[i][j][len(w[i][j])-2]==5):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g5_4(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [4]:
                a = a + [[4]]
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==4 and w[i][j][len(w[i][j])-2]>5):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g4_4(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [4]:
                a = a + [[4]]
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==4 and w[i][j][len(w[i][j])-2]>4):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_6_5_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]==5 and w[i][j][len(w[i][j])-3]==6):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_6_4_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]==4 and w[i][j][len(w[i][j])-3]==6):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_7_4_4(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==4 and w[i][j][len(w[i][j])-2]==4 and w[i][j][len(w[i][j])-3]==7):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g6_5_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [5,3]:
                a = a + [[5,3]]
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]==5 and w[i][j][len(w[i][j])-3]>6):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g7_4_4(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [4,4]:
                a = a + [[4,4]]
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==4 and w[i][j][len(w[i][j])-2]==4 and w[i][j][len(w[i][j])-3]>7):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g6_4_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [4,3]:
                a = a + [[4,3]]
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]==4 and w[i][j][len(w[i][j])-3]>6):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_4_4or6_5_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==4 and w[i][j][len(w[i][j])-2]==4):
                    a = a + [w[i][j]]
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]==5 and w[i][j][len(w[i][j])-3]==6):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_3_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==1 and w[i][j][len(w[i][j])-2]==3):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_5_3_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==1 and w[i][j][len(w[i][j])-2]==3 and w[i][j][len(w[i][j])-3]==5):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g5_3_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [3,1]:
                a = a + [[3,1]]
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==1 and w[i][j][len(w[i][j])-2]==3 and w[i][j][len(w[i][j])-3]>5):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g4_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [1]:
                a = a + [[1]]
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==1 and w[i][j][len(w[i][j])-2]>4):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_5_4_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==1 and w[i][j][len(w[i][j])-2]==4 and w[i][j][len(w[i][j])-3]==5):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g5_4_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [4,1]:
                a = a + [[4,1]]
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==1 and w[i][j][len(w[i][j])-2]==4 and w[i][j][len(w[i][j])-3]>5):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_4_1_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if number_of_ones(w[i][j])==2:
                if len(w[i][j])>2:
                    if w[i][j][len(w[i][j])-3]==4:
                        a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g4_1_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if number_of_ones(w[i][j])==2:
                if w[i][j]==[1,1]:
                    a = a + [[1,1]]
                if len(w[i][j])>2:
                    if w[i][j][len(w[i][j])-3]>4:
                        a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def check_leading_monomials(n):
    piv = list(newbase_subspace(n).pivots())
    return piv == missing_partitions_final(n)

def check_leading_monomials_up_to(n):
    for i in range(n+1):
        print([i,check_leading_monomials(i)])

PARI stack size set to 9589934592 bytes, maximum size set to 9589936128


In [5]:
newPartitions1(7)

[[2, 2, 2, 1],
 [2, 2, 1, 1, 1],
 [2, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [3, 2, 2],
 [3, 2, 1, 1],
 [3, 1, 1, 1, 1],
 [3, 3, 1],
 [4, 2, 1],
 [4, 1, 1, 1],
 [4, 3],
 [5, 2],
 [5, 1, 1],
 [6, 1],
 [7]]

In [6]:
newbase_subspace(7)

[       1        0        0        0        0        0        0        0        0        0   -75/64        0 -255/128    -15/8  -255/32]
[       0        1        0        0        0        0        0        0        0        0   -45/16        0  -153/32     -9/2   -153/8]
[       0        0        1        0        0        0        0        0        0        0    -75/4        0   -135/8      -30   -135/2]
[       0        0        0        1        0        0        0        0        0        0      -45        0   -225/2     -360     -450]
[       0        0        0        0        1        0        0        0        0        0   -15/16        0    -3/32     -3/2    -27/8]
[       0        0        0        0        0        1        0        0        0        0     -5/4        0     -1/8       -2     -9/2]
[       0        0        0        0        0        0        1        0        0        0        9        0     -3/2      -24      -30]
[       0        0        0        0     

In [ ]:
%time check_leading_monomials_up_to(31)

[0, True]
[1, True]
[2, True]
[3, True]
[4, True]
[5, True]
[6, True]
[7, True]
[8, True]
[9, True]
[10, True]
[11, True]
[12, True]
[13, True]
[14, True]
[15, True]
[16, True]
[17, True]
[18, True]
[19, True]
[20, True]
[21, True]
[22, True]
[23, True]
[24, True]
[25, True]
[26, True]
[27, True]
[28, True]
[29, True]
